In [6]:
cd "C:\Users\ohslo\Desktop\Manuscripts\Ongoing\Racial Capitalism and Neighborhood Health Disparities\Data"
use "Data_CA_Covid_imputed.dta", clear
sort id
ds


C:\Users\ohslo\Desktop\Manuscripts\Ongoing\Racial Capitalism and Neighborhood He
> alth Disparities\Data



id            gini          noinsurance   lnincome      _est_case_3
county        poverty       publicway     lnhome        _est_case_4
cases         unemployed    owner         wbratio       _est_case_5
death         noba          home          whratio       _est_case_6
lat           singlemom     income        _mi_m         _est_death_1
lon           assistance    doctor        _mi_id        _est_death_2
area          disadvan      doctor_ratio  _mi_miss      _est_death_3
total         density       pblack        imput_wbra~o  _est_death_4
white         male          phispanic     int1          _est_death_5
black         old           case_ratio    _est_case_1   _est_death_6
hispanic      five          death_ratio   _est_case_2


In [7]:
quietly {
    use "Data_CA_Covid_imputed.dta", clear
    keep lat lon    
    spatwmat, stand name(CA) xcoord(lat) ycoord(lon) band(0 1)
    svmat CA
    keep CA*
    save "CA", replace
    spmat dta CA CA*, replace
    spmat save CA using CA.spmat, replace
}


   Beware! 5 locations have no neighbors
   You are advised to extend the distance band


In [8]:
use "Data_CA_Covid_imputed.dta"
  spatgsa case_ratio, weights(CA) moran
  spatgsa death_ratio, weights(CA) moran





Measures of global spatial autocorrelation


Weights matrix
--------------------------------------------------------------
Name: CA
Type: Distance-based (inverse distance)
Distance band: 0.0 < d <= 1.0
Row-standardized: Yes
--------------------------------------------------------------

Moran's I
--------------------------------------------------------------
          Variables |    I      E(I)   sd(I)     z    p-value*
--------------------+-----------------------------------------
         case_ratio |  0.231  -0.018   0.112   2.227   0.013
--------------------------------------------------------------
*1-tail test





Measures of global spatial autocorrelation


Weights matrix
--------------------------------------------------------------
Name: CA
Type: Distance-based (inverse distance)
Distance band: 0.0 < d <= 1.0
Row-standardized: Yes
--------------------------------------------------------------

Moran's I
--------------------------------------------------------------
      

In [18]:
gen lnincome = ln(income-1)

In [22]:
gen lnhome = ln(home-1)

In [23]:
sum lnhome


    Variable |       Obs        Mean    Std. Dev.       Min        Max
-------------+--------------------------------------------------------
      lnhome |        58    12.86078    .4934905    11.9421    13.9666


In [152]:
save, replace

file Data_CA_Covid_imputed.dta saved


In [9]:
gen int1 = wbratio * whratio

int1 already defined


r(110);


In [24]:
quietly {
     use "Data_CA_Covid_imputed.dta"
  spreg ml case_ratio wbratio poverty density old doctor_ratio, elmat(CA) dlmat(CA) id(id)  
  estimates store case_1
  spreg ml case_ratio whratio poverty density old doctor_ratio, elmat(CA) dlmat(CA) id(id)  
  estimates store case_2 
  spreg ml case_ratio wbratio poverty whratio int1 density old doctor_ratio, elmat(CA) dlmat(CA) id(id)  
  estimates store case_3 
}    

In [35]:
%html
esttab case_1 case_2 case_3, ///
  cells(b(star fmt(3)) se(fmt(3) par)) beta starlevels(* .05 ** .01 *** .001) legend /// 
  stats(ll N, labels("Log Likelihood" "N") fmt(%10.2fc %10.0fc)) title("Table 2. Covid cases and Racial Income Inequality") ///
  mlabels("Black" "Hispanic" "Interaction") ///
  keep(wbratio whratio int1 poverty density old doctor_ratio rho:_cons) ///
  order(wbratio whratio int1 poverty density old doctor_ratio rho:_cons) ///
  varlabels(wbratio "White-Black income ratio" whratio "White-Hispanic income ratio" int1 "Interaction" ///
  density "Population density" old "Percent old" doctor_ratio "Healthcare professions" ///
  poverty "Poverty rate" rho:_cons "Spatial lag") eqlabels(, none) ///
html

This front-end or document format cannot display HTML

In [38]:
%html
esttab case_1 case_2 case_3, ///
  cells(beta(star fmt(3)) se(fmt(3) par)) starlevels(* .05 ** .01 *** .001) legend /// 
  stats(ll N, labels("Log Likelihood" "N") fmt(%10.2fc %10.0fc)) title("Table 2. Covid cases and Racial Income Inequality") ///
  mlabels("Black" "Hispanic" "Interaction") ///
  keep(wbratio whratio int1 poverty density old doctor_ratio rho:_cons) ///
  order(wbratio whratio int1 poverty density old doctor_ratio rho:_cons) ///
  varlabels(wbratio "White-Black income ratio" whratio "White-Hispanic income ratio" int1 "Interaction" ///
  density "Population density" old "Percent old" doctor_ratio "Healthcare professions" ///
  poverty "Poverty rate" rho:_cons "Spatial lag") eqlabels(, none) ///
html

This front-end or document format cannot display HTML

In [30]:
sum case_ratio wbratio whratio poverty density old doctor_ratio, d


                         case_ratio
-------------------------------------------------------------
      Percentiles      Smallest
 1%     8.034611       8.034611
 5%      12.6671       12.55285
10%      21.3268        12.6671       Obs                  58
25%     31.07909        14.9578       Sum of Wgt.          58

50%     41.89987                      Mean           47.79506
                        Largest       Std. Dev.      25.18906
75%     62.21626       91.26791
90%     75.47866       105.4326       Variance       634.4886
95%     105.4326       125.1739       Skewness       1.254123
99%     132.6001       132.6001       Kurtosis       5.078594

                           wbratio
-------------------------------------------------------------
      Percentiles      Smallest
 1%     .2944942       .2944942
 5%     .8670563       .7247237
10%     .9516871       .8670563       Obs                  58
25%     1.180058       .9064937       Sum of Wgt.          58

50%     1.441393   

In [28]:
pwcorr case_ratio wbratio whratio poverty density old doctor_ratio, sig


             | case_r~o  wbratio  whratio  poverty  density      old doctor~o
-------------+---------------------------------------------------------------
  case_ratio |   1.0000 
             |
             |
     wbratio |   0.0274   1.0000 
             |   0.8381
             |
     whratio |   0.2835   0.1662   1.0000 
             |   0.0311   0.2124
             |
     poverty |   0.2230  -0.1639  -0.2553   1.0000 
             |   0.0924   0.2190   0.0531
             |
     density |  -0.1125  -0.0149   0.1512  -0.1917   1.0000 
             |   0.4004   0.9117   0.2574   0.1495
             |
         old |  -0.5838   0.4094  -0.0610  -0.1542  -0.1354   1.0000 
             |   0.0000   0.0014   0.6492   0.2479   0.3108
             |
doctor_ratio |  -0.2937  -0.2284   0.0884  -0.5120   0.3114  -0.0385   1.0000 
             |   0.0252   0.0847   0.5093   0.0000   0.0173   0.7740
             |


In [29]:
quietly reg case_ratio wbratio whratio poverty density old doctor_ratio  
  vif




    Variable |       VIF       1/VIF  
-------------+----------------------
doctor_ratio |      1.68    0.594599
     poverty |      1.61    0.622050
     wbratio |      1.44    0.695349
         old |      1.28    0.780911
     density |      1.16    0.865023
     whratio |      1.13    0.884147
-------------+----------------------
    Mean VIF |      1.38


In [11]:
esttab case_1 case_2 case_3 case_4 case_5 case_6 using "table2.rtf", ///
  cells(b(star fmt(3)) se(fmt(3) par)) starlevels(* .05 ** .01 *** .001) legend /// 
  stats(ll N, labels("Log Likelihood" "N") fmt(%10.2fc %10.0fc)) title("Table 2. Covid cases and Racial Income Inequality") ///
  mlabels("Black_base" "Black_full" "Hispanic_base" "Hispanic_full" "Interaction") ///
  keep(wbratio whratio int1 _cons) order(wbratio whratio int1 _cons) ///
  varlabels(case_ratio:_cons "Constant" rho:_cons "Spatial lag" /// 
  wbratio "White-Black income ratio" whratio "White-Hispanic income ratio" int1 "Interaction") ///
  drop(lambda:_cons sigma2:_cons)  eqlabels(, none) ///
rtf

(output written to table2.rtf)


In [31]:
save, replace

file Data_CA_Covid_imputed.dta saved
